## Introduction: Physcian Conversion Model

**An HCP conversion model aims to predict the likelihood of a healthcare provider (HCP) becoming a writer for a particular company or brand for the first time. It helps pharmaceutical or medical device companies identify potential HCPs who have not yet written prescriptions for their products but may be receptive to doing so in the future. The model focuses on converting non-writing HCPs into active writers**

Here's a detailed view of an HCP conversion model:

1. Target Variable: The target variable in an HCP conversion model is binary, indicating whether an HCP becomes a writer (1) or not (0) for a specific company or brand. This variable serves as the outcome to be predicted.

2. Predictive Features: Various data sources and types can be used to develop an HCP conversion model. Some common predictive features include:

	  - Demographic Data: Information about the HCP's age, gender, specialty, location, education level, and other relevant characteristics. Demographic data helps identify patterns and preferences specific to certain groups of HCPs.
	
	  - Claims Data: Historical claims data can provide insights into the HCP's prescribing behavior, utilization patterns, and therapeutic areas of interest. It helps identify HCPs with similar patient profiles or conditions that align with the company's products.
	
	  - Promotional Data: Data on past promotional activities directed at the HCP, such as detailing visits, samples provided, speaker programs attended, and engagement with educational materials. This helps assess the level of previous interaction and the impact of promotional efforts.
	
	  - Network Analysis: Examination of the HCP's professional network, affiliations, collaborations, and referral patterns. Understanding the relationships between HCPs can help identify opinion leaders and assess the impact of peer influence.

**Note: Other types of Features can also be utilized depending on availability and Granularity.**

### Loading Required Libraries and Dependencies

In [1]:
#System and dataframe dependencies
import pandas as pd
import numpy as np
import os
import sys
import warnings
warnings.filterwarnings("ignore")

# Visual libraries
import matplotlib.pyplot as plt

# Importing necessary libraries for encoding
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#Sagemaker dependencies
import boto3
from io import BytesIO
import sagemaker

import time
from time import gmtime, strftime, sleep
import datetime

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### Import the dataset and clean column names

In [2]:
# Initialize the S3 client
s3_client = boto3.client('s3')

# Define the S3 bucket and file path
bucket_name = 'sagemaker-experiment-hs'
file_key = 'pharma-classifier-sample-data/Input_data.csv'

# Read the file directly from S3 into a DataFrame
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
df_input = pd.read_csv(BytesIO(response['Body'].read()))

#Clean column names
df_input.columns = df_input.columns.str.strip()
df_input.columns = df_input.columns.str.replace(' ', '_')

# Display the first few rows of the loaded data
df_input.head()

,NPI_ID,HCP_ID,TARGET,Age,Sex,Specialty,Year_of_Experience,HCO_Affiliation,HCO_Affiliation_Type,Number_of_Rx,...,F2F_visit,F2F_visit_last_1_month,F2F_visit_last_3_month,F2F_visit_last_6_month,F2F_visit_last_12_month,VRC_visit,VRC_visit_last_1_month,VRC_visit_last_3_month,VRC_visit_last_6_month,VRC_visit_last_12_month
0,9846255,HCP_1,0,64,M,Oncology,10,XYZ Medical Center,Referral,290,...,1,2,3,5,9,2,3,5,9,16
1,5217093,HCP_2,1,64,F,Neuro-oncology,47,ABC Medical Center,Collaboration,645,...,2,4,6,8,12,1,2,3,6,11
2,2659257,HCP_3,1,56,M,Hematology,41,STU Medical Center,Employment,879,...,1,2,3,4,6,2,3,6,8,11
3,9851763,HCP_4,1,74,F,Pediatric,35,XYZ Medical Center,Collaboration,392,...,1,2,4,8,15,1,2,4,6,7
4,6650853,HCP_5,0,46,M,Immunology,60,DEF Medical Center,Contract,1043,...,1,2,3,5,8,2,4,6,9,16


### General EDA and Data checks

In [3]:
#General EDA
pd.options.display.max_columns = 50
print()
df_input.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 59 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   NPI_ID                              5000 non-null   int64 
 1   HCP_ID                              5000 non-null   object
 2   TARGET                              5000 non-null   int64 
 3   Age                                 5000 non-null   int64 
 4   Sex                                 5000 non-null   object
 5   Specialty                           5000 non-null   object
 6   Year_of_Experience                  5000 non-null   int64 
 7   HCO_Affiliation                     5000 non-null   object
 8   HCO_Affiliation_Type                5000 non-null   object
 9   Number_of_Rx                        5000 non-null   int64 
 10  Rx_last_1_Month                     5000 non-null   int64 
 11  Rx_last_3_Month                     5000 non-null   int

In [4]:
df_input.describe()

,NPI_ID,TARGET,Age,Year_of_Experience,Number_of_Rx,Rx_last_1_Month,Rx_last_3_Month,Rx_last_6_Month,Rx_last_12_Month,Number_of_Px,Px_last_1_Month,Px_last_3_Month,Px_last_6_Month,Px_last_12_Month,Claims_last_1_Month,Claims_last_3_Month,Claims_last_6_Month,Claims_last_12_Month,Procedures_chemo_last_1_month,Procedures_chemo_last_3_month,Procedures_chemo_last_6_month,Procedures_chemo_last_12_month,Procedures_radio_last_1_month,Procedures_radio_last_3_month,Procedures_radio_last_6_month,...,Procedures_Immuno_last_12_month,Procedures_Biopsy_last_1_month,Procedures_Biopsy_last_3_month,Procedures_Biopsy_last_6_month,Procedures_Biopsy_last_12_month,Promotional_doximity,Promotional_doximity_last_1_month,Promotional_doximity_last_3_month,Promotional_doximity_last_6_month,Promotional_doximity_last_12_month,Promotional_medscape,Promotional_medscape_last_1_month,Promotional_medscape_last_3_month,Promotional_medscape_last_6_month,Promotional_medscape_last_12_month,F2F_visit,F2F_visit_last_1_month,F2F_visit_last_3_month,F2F_visit_last_6_month,F2F_visit_last_12_month,VRC_visit,VRC_visit_last_1_month,VRC_visit_last_3_month,VRC_visit_last_6_month,VRC_visit_last_12_month
count,5.000000e+03,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.00000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000
mean,5.544598e+06,0.250200,60.198200,36.400000,794.586800,1186.889600,1773.687600,2668.695800,4001.142200,183.946000,276.215400,414.737600,620.811400,929.16340,178.329600,268.634400,404.673600,608.417000,49.09620,74.523400,112.546000,168.131200,57.620800,86.995200,130.363800,...,197.662400,57.33700,86.49120,130.503800,196.412800,7.510600,11.781400,18.219800,27.73620,42.021800,7.45780,11.677400,18.056400,27.510000,41.669000,1.502000,2.749000,4.627600,7.436600,11.724200,1.50000,2.748400,4.625800,7.448800,11.645400
std,2.589635e+06,0.433171,18.770258,18.600312,327.511269,546.032458,899.699968,1464.578381,2357.893309,66.730701,114.531764,192.461137,316.462382,517.74319,98.029741,159.358798,256.020525,406.822833,15.36382,27.539425,47.387089,78.760197,13.913408,27.094509,47.897752,...,82.541647,13.84632,26.46427,48.327858,82.397703,1.715948,3.323947,6.164665,10.78209,18.106466,1.71266,3.363211,6.118718,10.584078,18.069044,0.500046,0.825915,1.459638,2.608091,4.587281,0.50005,0.827789,1.463763,2.621899,4.520105
min,1.001849e+06,0.000000,28.000000,10.000000,100.000000,102.000000,106.000000,112.000000,126.000000,70.000000,73.000000,79.000000,91.000000,116.00000,8.000000,9.000000,12.000000,15.000000,23.00000,24.000000,26.000000,32.000000,34.000000,35.000000,38.000000,...,48.000000,34.00000,35.00000,36.000000,46.000000,5.000000,6.000000,7.000000,8.00000,9.000000,5.00000,6.000000,7.000000,8.000000,9.000000,1.000000,2.000000,3.000000,4.000000,5.000000,1.00000,2.000000,3.000000,4.000000,5.000000
25%,3.326632e+06,0.000000,44.000000,17.000000,525.000000,760.000000,1078.000000,1553.000000,2227.750000,126.000000,182.000000,259.000000,384.000000,545.00000,95.000000,136.000000,197.000000,287.750000,35.00000,52.000000,75.000000,109.000000,45.000000,66.000000,95.000000,...,136.000000,45.00000,66.00000,95.000000,135.000000,6.000000,9.000000,14.000000,20.00000,29.000000,6.00000,9.000000,13.000000,20.000000,28.000000,1.000000,2.000000,4.000000,5.000000,8.000000,1.00000,2.000000,4.000000,5.000000,8.000000
50%,5.601028e+06,0.000000,60.000000,40.000000,924.000000,1224.000000,1756.000000,2549.500000,3701.000000,184.000000,266.000000,383.000000,557.000000,808.50000,177.000000,258.000000,371.500000,544.000000,49.00000,7

In [5]:
#Check for categorical columns
df_input.select_dtypes('object').nunique()

HCP_ID                  5000
Sex                        2
Specialty                  6
HCO_Affiliation           10
HCO_Affiliation_Type       4
dtype: int64

In [6]:
#Drop unwanted column: "HCO Affiliation" - "Affiliation Type" is more valid column for us
df_input.drop(['HCO_Affiliation'], axis= 1, inplace= True)
print()
print(df_input.select_dtypes('object').nunique())


HCP_ID                  5000
Sex                        2
Specialty                  6
HCO_Affiliation_Type       4
dtype: int64


### Dummy Encode Required Variables

In [7]:
onehot_cols = ['Sex', 'Specialty', 'HCO_Affiliation_Type']

In [8]:
df_input = pd.get_dummies(df_input, columns=onehot_cols, drop_first=True)
print(df_input.shape)
print('')
df_input.head()

(5000, 64)



,NPI_ID,HCP_ID,TARGET,Age,Year_of_Experience,Number_of_Rx,Rx_last_1_Month,Rx_last_3_Month,Rx_last_6_Month,Rx_last_12_Month,Number_of_Px,Px_last_1_Month,Px_last_3_Month,Px_last_6_Month,Px_last_12_Month,Claims_last_1_Month,Claims_last_3_Month,Claims_last_6_Month,Claims_last_12_Month,Procedures_chemo_last_1_month,Procedures_chemo_last_3_month,Procedures_chemo_last_6_month,Procedures_chemo_last_12_month,Procedures_radio_last_1_month,Procedures_radio_last_3_month,...,Promotional_doximity_last_12_month,Promotional_medscape,Promotional_medscape_last_1_month,Promotional_medscape_last_3_month,Promotional_medscape_last_6_month,Promotional_medscape_last_12_month,F2F_visit,F2F_visit_last_1_month,F2F_visit_last_3_month,F2F_visit_last_6_month,F2F_visit_last_12_month,VRC_visit,VRC_visit_last_1_month,VRC_visit_last_3_month,VRC_visit_last_6_month,VRC_visit_last_12_month,Sex_ M,Specialty_Immunology,Specialty_Neuro-oncology,Specialty_Oncology,Specialty_Pediatric,Specialty_Uro-oncology,HCO_Affiliation_Type_Contract,HCO_Affiliation_Type_Employment,HCO_Affiliation_Type_Referral
0,9846255,HCP_1,0,64,10,290,400,492,770,1373,234,464,738,1060,1949,241,395,699,1047,48,82,112,154,43,57,...,47,8,12,18,30,55,1,2,3,5,9,2,3,5,9,16,True,False,False,True,False,False,False,False,True
1,5217093,HCP_2,1,64,47,645,923,1628,2210,2449,80,139,264,318,534,197,266,420,667,66,75,98,124,59,99,...,23,6,11,14,21,40,2,4,6,8,12,1,2,3,6,11,False,False,True,False,False,False,False,False,False
2,2659257,HCP_3,1,56,41,879,1062,1918,3505,6169,197,274,386,642,826,319,532,655,849,37,55,96,129,78,141,...,37,10,17,20,29,48,1,2,3,4,6,2,3,6,8,11,True,False,False,False,False,False,False,True,False
3,9851763,HCP_4,1,74,35,392,704,1036,1078,1184,217,260,319,401,671,232,419,464,849,69,114,139,196,54,69,...,54,6,12,16,21,27,1,2,4,8,15,1,2,4,6,7,False,False,False,False,True,False,False,False,False
4,6650853,HCP_5,0,46,60,1043,1228,1305,1339,2483,209,269,456,591,642,232,261,519,930,75,83,110,163,55,90,...,26,8,12,15,30,52,1,2,3,5,8,2,4,6,9,16,True,True,False,False,False,False,True,False,False


In [9]:
# Clean column names by stripping spaces and replacing spaces between names
df_input.columns = df_input.columns.str.strip().str.replace(' ', '')
print(df_input.columns.tolist())

['NPI_ID', 'HCP_ID', 'TARGET', 'Age', 'Year_of_Experience', 'Number_of_Rx', 'Rx_last_1_Month', 'Rx_last_3_Month', 'Rx_last_6_Month', 'Rx_last_12_Month', 'Number_of_Px', 'Px_last_1_Month', 'Px_last_3_Month', 'Px_last_6_Month', 'Px_last_12_Month', 'Claims_last_1_Month', 'Claims_last_3_Month', 'Claims_last_6_Month', 'Claims_last_12_Month', 'Procedures_chemo_last_1_month', 'Procedures_chemo_last_3_month', 'Procedures_chemo_last_6_month', 'Procedures_chemo_last_12_month', 'Procedures_radio_last_1_month', 'Procedures_radio_last_3_month', 'Procedures_radio_last_6_month', 'Procedures_radio_last_12_month', 'Procedures_Immuno_last_1_month', 'Procedures_Immuno_last_3_month', 'Procedures_Immuno_last_6_month', 'Procedures_Immuno_last_12_month', 'Procedures_Biopsy_last_1_month', 'Procedures_Biopsy_last_3_month', 'Procedures_Biopsy_last_6_month', 'Procedures_Biopsy_last_12_month', 'Promotional_doximity', 'Promotional_doximity_last_1_month', 'Promotional_doximity_last_3_month', 'Promotional_doximity_l

### Creating a Feature Store
> A fully managed, purpose-built repository to store, share and manage features for Machine learning (ML) Models

In [10]:
# Convert boolean columns to string
bool_columns = df_input.select_dtypes(include=bool).columns
df_input[bool_columns] = df_input[bool_columns].astype(str)

df_input['NPI_ID'] = df_input['NPI_ID'].astype(str)

>> Setting System Parameters

In [12]:
from sagemaker.session import Session
boto_session = boto3.Session(region_name="us-east-1")
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
prefix = 'sagemaker-featurestore'

In [13]:
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

In [14]:
feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

In [15]:
offline_feature_store_bucket = 's3://*{}*/*{}*'.format(bucket_name, prefix)
print(offline_feature_store_bucket)

s3://*sagemaker-experiment-hs*/*sagemaker-featurestore*


#### Create a feature group

In [16]:
from sagemaker.feature_store.feature_group import FeatureGroup
feature_group_name = "pharma-conversion-classifier-feature-group"
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=feature_store_session)

In [17]:
import time
from time import gmtime, strftime, sleep
from sagemaker.feature_store.feature_group import FeatureGroup

physician_conversion_classifier_feature_group = 'physician-conversion-feature-group-' + strftime('%d-%H-%M-%S', gmtime())

physician_conversion_classifier_feature_group = FeatureGroup(name=physician_conversion_classifier_feature_group, sagemaker_session=feature_store_session)


In [18]:
# Creating Lookup column and date column 
record_identifier_name = "HCP_ID"
event_time_feature_name = "EventTime"
current_time_sec = int(round(time.time()))
df_input[event_time_feature_name] = pd.Series([current_time_sec]*len(df_input), dtype="float64")

In [19]:
physician_conversion_classifier_feature_group.load_feature_definitions(data_frame=df_input);

In [20]:
# Load feature definitions
feature_definitions = physician_conversion_classifier_feature_group.load_feature_definitions(data_frame=df_input)

# Print or display the feature definitions
print(feature_definitions)


[FeatureDefinition(feature_name='NPI_ID', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None), FeatureDefinition(feature_name='HCP_ID', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None), FeatureDefinition(feature_name='TARGET', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None), FeatureDefinition(feature_name='Age', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None), FeatureDefinition(feature_name='Year_of_Experience', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None), FeatureDefinition(feature_name='Number_of_Rx', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None), FeatureDefinition(feature_name='Rx_last_1_Month', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None), FeatureDefinition(feature_name='Rx_last_3_Month', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None), FeatureDefinition(feature_name='Rx_las

In [22]:
physician_conversion_classifier_feature_group.create(
    s3_uri= 's3://sagemaker-experiment-hs/Feature-store-trial/', #offline_feature_store_bucket,
    record_identifier_name=record_identifier_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=False
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:244134863681:feature-group/physician-conversion-feature-group-30-10-51-54',
 'ResponseMetadata': {'RequestId': '110a64ff-7818-44f8-ba89-ccd1e14ffb01',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '110a64ff-7818-44f8-ba89-ccd1e14ffb01',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '123',
   'date': 'Tue, 30 Apr 2024 10:52:37 GMT'},
  'RetryAttempts': 0}}

In [26]:
def check_feature_group_status(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group to be Created")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    print(f"FeatureGroup {feature_group.name} successfully created.")

check_feature_group_status(physician_conversion_classifier_feature_group)

FeatureGroup physician-conversion-feature-group-30-10-51-54 successfully created.


#### Load data to Feature Group

In [27]:
physician_conversion_classifier_feature_group.ingest(data_frame=df_input, max_workers=1, max_processes = 1, wait=True)

IngestionManagerPandas(feature_group_name='physician-conversion-feature-group-30-10-51-54', feature_definitions={'NPI_ID': {'FeatureName': 'NPI_ID', 'FeatureType': 'String'}, 'HCP_ID': {'FeatureName': 'HCP_ID', 'FeatureType': 'String'}, 'TARGET': {'FeatureName': 'TARGET', 'FeatureType': 'Integral'}, 'Age': {'FeatureName': 'Age', 'FeatureType': 'Integral'}, 'Year_of_Experience': {'FeatureName': 'Year_of_Experience', 'FeatureType': 'Integral'}, 'Number_of_Rx': {'FeatureName': 'Number_of_Rx', 'FeatureType': 'Integral'}, 'Rx_last_1_Month': {'FeatureName': 'Rx_last_1_Month', 'FeatureType': 'Integral'}, 'Rx_last_3_Month': {'FeatureName': 'Rx_last_3_Month', 'FeatureType': 'Integral'}, 'Rx_last_6_Month': {'FeatureName': 'Rx_last_6_Month', 'FeatureType': 'Integral'}, 'Rx_last_12_Month': {'FeatureName': 'Rx_last_12_Month', 'FeatureType': 'Integral'}, 'Number_of_Px': {'FeatureName': 'Number_of_Px', 'FeatureType': 'Integral'}, 'Px_last_1_Month': {'FeatureName': 'Px_last_1_Month', 'FeatureType': 'I

### Feature Selection
- Variance Threshold check
- Select k best for top n features 

In [28]:
id_target_col_list = ['NPI_ID', 'HCP_ID', 'TARGET']
col_for_feature_selection = df_input.columns.difference(id_target_col_list)

In [30]:
df_input[bool_columns] = df_input[bool_columns].astype(bool)

In [31]:
from sklearn.feature_selection import VarianceThreshold

var_thr = VarianceThreshold(threshold = 0.1) #Removing both constant and quasi-constant
var_thr.fit(df_input[col_for_feature_selection])

var_thr.get_support()

array([ True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False,  True,  True,  True,  True,  True,  True])

In [32]:
df_input_subset = df_input[col_for_feature_selection]
df_input_subset.columns

Index(['Age', 'Claims_last_12_Month', 'Claims_last_1_Month',
       'Claims_last_3_Month', 'Claims_last_6_Month', 'EventTime', 'F2F_visit',
       'F2F_visit_last_12_month', 'F2F_visit_last_1_month',
       'F2F_visit_last_3_month', 'F2F_visit_last_6_month',
       'HCO_Affiliation_Type_Contract', 'HCO_Affiliation_Type_Employment',
       'HCO_Affiliation_Type_Referral', 'Number_of_Px', 'Number_of_Rx',
       'Procedures_Biopsy_last_12_month', 'Procedures_Biopsy_last_1_month',
       'Procedures_Biopsy_last_3_month', 'Procedures_Biopsy_last_6_month',
       'Procedures_Immuno_last_12_month', 'Procedures_Immuno_last_1_month',
       'Procedures_Immuno_last_3_month', 'Procedures_Immuno_last_6_month',
       'Procedures_chemo_last_12_month', 'Procedures_chemo_last_1_month',
       'Procedures_chemo_last_3_month', 'Procedures_chemo_last_6_month',
       'Procedures_radio_last_12_month', 'Procedures_radio_last_1_month',
       'Procedures_radio_last_3_month', 'Procedures_radio_last_6_month'

In [33]:
remove_col_list = [col for col in df_input_subset.columns 
          if col not in df_input_subset.columns[var_thr.get_support()]]

remove_col_list

['EventTime',
 'HCO_Affiliation_Type_Contract',
 'HCO_Affiliation_Type_Employment',
 'HCO_Affiliation_Type_Referral',
 'Sex_M',
 'Specialty_Immunology',
 'Specialty_Neuro-oncology',
 'Specialty_Oncology',
 'Specialty_Pediatric',
 'Specialty_Uro-oncology']

### Feature Selection Using Select K Best

In [34]:
#Select top n Features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

def select_kbest_features(df, target_col,n):
  """
  Selects the top n features from the DataFrame using the SelectKBest algorithm.

  Args:
    df: The DataFrame to select features from.
    n: The number of features to select.

  Returns:
    A list of the top n features.
  """


  selector = SelectKBest(k=n)
  selected_features = selector.fit_transform(df, target_col)
  
  mask = selector.get_support()
  top_n_features = df.columns[mask]

  return top_n_features

In [35]:
id_col_list = ['NPI_ID', 'HCP_ID']
target_col = df_input['TARGET']
top_n_col_list = select_kbest_features(df_input.drop(id_col_list,axis=1),target_col, 30)
print(len(top_n_col_list))
top_n_col_list

30


Index(['TARGET', 'Age', 'Year_of_Experience', 'Number_of_Rx',
       'Rx_last_1_Month', 'Rx_last_3_Month', 'Rx_last_6_Month',
       'Rx_last_12_Month', 'Number_of_Px', 'Px_last_3_Month',
       'Claims_last_1_Month', 'Claims_last_3_Month', 'Claims_last_6_Month',
       'Claims_last_12_Month', 'Procedures_chemo_last_1_month',
       'Procedures_chemo_last_3_month', 'Procedures_chemo_last_6_month',
       'Procedures_chemo_last_12_month', 'Procedures_radio_last_6_month',
       'Procedures_radio_last_12_month', 'Procedures_Biopsy_last_6_month',
       'Procedures_Biopsy_last_12_month', 'Promotional_doximity',
       'Promotional_medscape', 'Promotional_medscape_last_1_month',
       'Promotional_medscape_last_3_month',
       'Promotional_medscape_last_6_month',
       'Promotional_medscape_last_12_month', 'VRC_visit_last_6_month',
       'VRC_visit_last_12_month'],
      dtype='object')

In [36]:
#Convert to list
top_n_col_list = top_n_col_list.tolist()

type(top_n_col_list)

list

In [ ]:
cols_for_model_df_list = id_col_list + top_n_col_list
print(len(cols_for_model_df_list))
print('')

In [37]:
cols_for_model_df_list = id_col_list + top_n_col_list
print(len(cols_for_model_df_list))
print('')

32



In [38]:
df_feature_eng_output = df_input[cols_for_model_df_list]
df_feature_eng_output.head()

,NPI_ID,HCP_ID,TARGET,Age,Year_of_Experience,Number_of_Rx,Rx_last_1_Month,Rx_last_3_Month,Rx_last_6_Month,Rx_last_12_Month,Number_of_Px,Px_last_3_Month,Claims_last_1_Month,Claims_last_3_Month,Claims_last_6_Month,Claims_last_12_Month,Procedures_chemo_last_1_month,Procedures_chemo_last_3_month,Procedures_chemo_last_6_month,Procedures_chemo_last_12_month,Procedures_radio_last_6_month,Procedures_radio_last_12_month,Procedures_Biopsy_last_6_month,Procedures_Biopsy_last_12_month,Promotional_doximity,Promotional_medscape,Promotional_medscape_last_1_month,Promotional_medscape_last_3_month,Promotional_medscape_last_6_month,Promotional_medscape_last_12_month,VRC_visit_last_6_month,VRC_visit_last_12_month
0,9846255,HCP_1,0,64,10,290,400,492,770,1373,234,738,241,395,699,1047,48,82,112,154,79,110,182,209,8,8,12,18,30,55,9,16
1,5217093,HCP_2,1,64,47,645,923,1628,2210,2449,80,264,197,266,420,667,66,75,98,124,111,128,73,93,8,6,11,14,21,40,6,11
2,2659257,HCP_3,1,56,41,879,1062,1918,3505,6169,197,386,319,532,655,849,37,55,96,129,238,355,107,184,10,10,17,20,29,48,8,11
3,9851763,HCP_4,1,74,35,392,704,1036,1078,1184,217,319,232,419,464,849,69,114,139,196,91,177,133,262,6,6,12,16,21,27,6,7
4,6650853,HCP_5,0,46,60,1043,1228,1305,1339,2483,209,456,232,261,519,930,75,83,110,163,122,158,234,284,6,8,12,15,30,52,9,16


In [39]:
df_model_input = df_feature_eng_output.copy()

In [40]:

# Initialize the S3 client
s3_client = boto3.client('s3')

# Define the S3 bucket name and new folder path
bucket_name = 'sagemaker-experiment-hs'
new_folder_key = 'processed_output/'
file_key = 'pharma-classifier-sample-data/model_input.csv'  # Assuming this is the key for your processed file

# Save DataFrame to CSV file
df_model_input.to_csv('model_input.csv', index=False)

# Upload the file to S3 under the new folder
s3_client.upload_file('model_input.csv', bucket_name, new_folder_key + 'model_input.csv')

# Remove the local CSV file after uploading to S3
os.remove('model_input.csv')

# Confirm successful upload
print(f'File uploaded to s3://{bucket_name}/{new_folder_key}model_input.csv')


File uploaded to s3://sagemaker-experiment-hs/processed_output/model_input.csv
